In [0]:
 %tensorflow_version 1.x
import tensorflow as tf
from keras.applications import inception_v3
from keras.layers import Dense, Flatten, Activation, Dropout
from keras.optimizers import Adam,SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing import image
from keras.models import Model

In [0]:
inceptionv3_model = inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(224,224,3),
                                            classes=2)
x = inceptionv3_model.output
x = Flatten()(x)
x = Dense(units=256, activation='relu', name='fc1', kernel_initializer='he_normal')(x)
x = Dropout(rate=0.2)(x)
predictions = Dense(units=2, activation='softmax')(x)
model = Model(inputs=inceptionv3_model.input, outputs=predictions)

In [9]:
# 图像增强
train_datagen = image.ImageDataGenerator()
val_datagen = image.ImageDataGenerator()

batch_size= 32
train_generator = train_datagen.flow_from_directory(
    './image/train',
    target_size=(224,224),batch_size=batch_size
                                                   )
val_generator = val_datagen.flow_from_directory(
    './image/test',
    target_size=(224,224),batch_size=batch_size
                                               )

print(train_generator.class_indices)

Found 400 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
{'cat': 0, 'dog': 1}


In [10]:
earlystopping = EarlyStopping(monitor='val_loss', patience=9, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1,
                             factor=0.2, mode='auto', min_delta=0.0001, min_lr=1e-7)
model.compile(optimizer=SGD(lr=0.0001,momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=len(train_generator), epochs=201, validation_data=val_generator, 
                    validation_steps=len(val_generator),callbacks=[earlystopping,reduce_lr])

Epoch 1/201
13/13 [==============================] - 31s 2s/step - loss: 0.7876 - acc: 0.5694 - val_loss: 0.5146 - val_acc: 0.7600
Epoch 2/201
13/13 [==============================] - 10s 777ms/step - loss: 0.3714 - acc: 0.8463 - val_loss: 0.3447 - val_acc: 0.8600
Epoch 3/201
13/13 [==============================] - 10s 776ms/step - loss: 0.1905 - acc: 0.9351 - val_loss: 0.2950 - val_acc: 0.8850
Epoch 4/201
13/13 [==============================] - 10s 780ms/step - loss: 0.1330 - acc: 0.9662 - val_loss: 0.2708 - val_acc: 0.8900
Epoch 5/201
13/13 [==============================] - 10s 779ms/step - loss: 0.0894 - acc: 0.9831 - val_loss: 0.2628 - val_acc: 0.9050
Epoch 6/201
13/13 [==============================] - 10s 785ms/step - loss: 0.0907 - acc: 0.9788 - val_loss: 0.2672 - val_acc: 0.9000
Epoch 7/201
13/13 [==============================] - 10s 780ms/step - loss: 0.0643 - acc: 0.9879 - val_loss: 0.2589 - val_acc: 0.9050
Epoch 8/201
13/13 [==============================] - 10s 779ms/st